This session is for storing the basic operations when we do some data preparation work, please use the order_example.csv to see how it works 

In [125]:
import pandas as pd
import numpy as np

## Load data

In [106]:
df_order = pd.read_csv('order_example.csv')
df_order.head(5)

,门店,store_name,商品,material_name,日期,qty,unit
0,1124,北京二十四店,A0000127,小郡肝,2019.03.01,NaN,KG
1,1124,北京二十四店,3001455,巴沙鱼片,2019.03.01,30.0,KG
2,1124,北京二十四店,NaN,鲜贝滑,2019.03.01,1.5,KG
3,1124,北京二十四店,3012098,捞派脆毛肚,2019.03.01,16.0,KG
4,1124,北京二十四店,3012811,脆鸭郡把,2019.03.01,3.0,KG


## Rename the columns

In [107]:
df_order = df_order.rename(columns={'门店': 'store_id', '商品': 'sku_id', '日期': 'date','种类': 'catagory', 
                                   '数量': 'qty'})

## Delete missing value

In [108]:
df_order = df_order[~df_order['sku_id'].isnull()]
df_order.head(5)

,store_id,store_name,sku_id,material_name,date,qty,unit
0,1124,北京二十四店,A0000127,小郡肝,2019.03.01,NaN,KG
1,1124,北京二十四店,3001455,巴沙鱼片,2019.03.01,30.0,KG
3,1124,北京二十四店,3012098,捞派脆毛肚,2019.03.01,16.0,KG
4,1124,北京二十四店,3012811,脆鸭郡把,2019.03.01,3.0,KG
5,1124,北京二十四店,A0000162,虾滑,2019.03.01,1.0,KG


## Fill in missing data

In [109]:
df_order['qty'] = df_order['qty'].fillna(0)
df_order.head(5)

,store_id,store_name,sku_id,material_name,date,qty,unit
0,1124,北京二十四店,A0000127,小郡肝,2019.03.01,0.0,KG
1,1124,北京二十四店,3001455,巴沙鱼片,2019.03.01,30.0,KG
3,1124,北京二十四店,3012098,捞派脆毛肚,2019.03.01,16.0,KG
4,1124,北京二十四店,3012811,脆鸭郡把,2019.03.01,3.0,KG
5,1124,北京二十四店,A0000162,虾滑,2019.03.01,1.0,KG


## Set the type of data

In [110]:
df_order['store_id'] = df_order['store_id'].astype(str)
df_order['sku_id'] = df_order['sku_id'].astype(str)
df_order['date'] = pd.to_datetime(df_order['date'])

## Sum

In [111]:
print(len(df_order))
df_order = df_order.groupby(['store_id', 'sku_id', 'date'])['qty'].sum().reset_index()
df_order

132681


,store_id,sku_id,date,qty
0,1122,1000012,2019-03-02,15.0
1,1122,1000012,2019-03-03,15.0
2,1122,1000012,2019-03-06,15.0
3,1122,1000012,2019-03-09,15.0
4,1122,1000012,2019-03-11,15.0
...,...,...,...,...
114358,BJ17,A0000163,2019-08-25,17.5
114359,BJ17,A0000163,2019-08-27,7.5
114360,BJ17,A0000163,2019-08-28,7.5
114361,BJ17,A0000163,2019-08-29,17.5


## Sort order

In [112]:
df_order = df_order.sort_values(['store_id', 'sku_id', 'date'])
df_order.head(5)

,store_id,sku_id,date,qty
0,1122,1000012,2019-03-02,15.0
1,1122,1000012,2019-03-03,15.0
2,1122,1000012,2019-03-06,15.0
3,1122,1000012,2019-03-09,15.0
4,1122,1000012,2019-03-11,15.0


## Add new columns

In [113]:
df_order['idx'] = df_order['store_id'] + '_' + df_order['sku_id']
df_order.head(5)

,store_id,sku_id,date,qty,idx
0,1122,1000012,2019-03-02,15.0,1122_1000012
1,1122,1000012,2019-03-03,15.0,1122_1000012
2,1122,1000012,2019-03-06,15.0,1122_1000012
3,1122,1000012,2019-03-09,15.0,1122_1000012
4,1122,1000012,2019-03-11,15.0,1122_1000012


## Calculate time interval

In [120]:
order_move = pd.DataFrame()
for i in df_order['idx'].unique():
    temp = df_order[df_order['idx'] == i]
    temp = temp.sort_values(['date'])
    temp['last_date'] = temp['date'].shift(1)
    order_move = order_move.append(temp)
order_move['date'] = pd.to_datetime(order_move['date'])
order_move['last_date'] = pd.to_datetime(order_move['last_date'])
order_move['interval'] = (order_move['date'] - order_move['last_date']).dt.days
order_move.head(5)

,store_id,sku_id,date,qty,idx,last_date,interval
0,1122,1000012,2019-03-02,15.0,1122_1000012,NaT,NaN
1,1122,1000012,2019-03-03,15.0,1122_1000012,2019-03-02,1.0
2,1122,1000012,2019-03-06,15.0,1122_1000012,2019-03-03,3.0
3,1122,1000012,2019-03-09,15.0,1122_1000012,2019-03-06,3.0
4,1122,1000012,2019-03-11,15.0,1122_1000012,2019-03-09,2.0


## Calculate order times, average interval between orders and average qty

In [126]:
order_copy = df_order.copy()
order_copy['times'] = 1
order_copy['date'] = pd.to_datetime(order_copy['date'], format="%Y-%m-%d")
order_copy = order_copy.groupby(['store_id', 'sku_id']).agg(
    {'date': ['min', 'max'], 'times': np.sum, 'qty': np.sum}).reset_index()
order_copy.columns = ['store_id', 'sku_id', 'min_date', 'max_date', 'times', 'qty']
order_copy['diff'] = (pd.to_datetime(order_copy['max_date']) - pd.to_datetime(order_copy['min_date'])).dt.days + 1
order_copy['avg_interval'] = order_copy['diff'] / order_copy['times']
order_copy["avg_order_qty"] = order_copy["qty"] / order_copy["times"]
order_copy

,store_id,sku_id,min_date,max_date,times,qty,diff,avg_interval,avg_order_qty
0,1122,1000012,2019-03-02,2019-08-31,81,1410.0,183,2.259259,17.407407
1,1122,1000110,2019-03-02,2019-08-14,62,257.3,166,2.677419,4.150000
2,1122,1000137,2019-03-02,2019-06-10,43,7000.0,101,2.348837,162.790698
3,1122,1000198,2019-03-01,2019-08-31,159,3702.0,184,1.157233,23.283019
4,1122,1000201,2019-03-01,2019-08-31,82,444.0,184,2.243902,5.414634
...,...,...,...,...,...,...,...,...,...
1417,BJ17,A0000159,2019-03-01,2019-08-31,163,2628.0,184,1.128834,16.122699
1418,BJ17,A0000160,2019-03-01,2019-08-31,163,7403.5,184,1.128834,45.420245
1419,BJ17,A0000161,2019-03-02,2019-08-30,79,85.0,182,2.303797,1.075949
1420,BJ17,A0000162,2019-03-01,2019-08-31,163,4833.5,184,1.128834,29.653374


## Continue time 
(Rule: according to the minimum and maximum dates among all stores in the history of the product.)

In [127]:
datelist = df_order.groupby(['sku_id', 'store_id', 'idx']).agg({'date': ['min', 'max']}).reset_index()
datelist.columns = ['sku_id', 'store_id', 'idx', 'min_date', 'max_date']
datelist.head(5)

,sku_id,store_id,idx,min_date,max_date
0,1000012,1122,1122_1000012,2019-03-02,2019-08-31
1,1000012,1124,1124_1000012,2019-03-01,2019-08-31
2,1000012,1138,1138_1000012,2019-03-01,2019-08-31
3,1000012,1158,1158_1000012,2019-04-18,2019-08-31
4,1000012,1198,1198_1000012,2019-03-01,2019-08-31


In [128]:
datelist["minmin_date"] = datelist.groupby(["sku_id"])["min_date"].transform(min)
datelist["maxmax_date"] = datelist.groupby(["sku_id"])["max_date"].transform(max)
del datelist["min_date"]
del datelist["max_date"]    
datelist.head(5)

,sku_id,store_id,idx,minmin_date,maxmax_date
0,1000012,1122,1122_1000012,2019-03-01,2019-08-31
1,1000012,1124,1124_1000012,2019-03-01,2019-08-31
2,1000012,1138,1138_1000012,2019-03-01,2019-08-31
3,1000012,1158,1158_1000012,2019-03-01,2019-08-31
4,1000012,1198,1198_1000012,2019-03-01,2019-08-31


In [129]:
datelist = pd.melt(
    datelist,
    id_vars=["sku_id", "store_id", "idx"],
    value_name="date"
)
del datelist["variable"]
datelist['date'] = pd.to_datetime(datelist['date'])
datelist.set_index("date", inplace=True)
datelist["tmp"] = 0
datelist = datelist.groupby(["sku_id", "store_id", "idx"])["tmp"].resample('D').sum().reset_index()
del datelist["tmp"]
datelist.head(5)

,sku_id,store_id,idx,date
0,1000012,1122,1122_1000012,2019-03-01
1,1000012,1122,1122_1000012,2019-03-02
2,1000012,1122,1122_1000012,2019-03-03
3,1000012,1122,1122_1000012,2019-03-04
4,1000012,1122,1122_1000012,2019-03-05


In [130]:
df_final = pd.merge(datelist, df_order, on=['store_id', 'sku_id', 'date','idx'], how = "left")
df_final['qty'].fillna(0, inplace=True)
df_final

,sku_id,store_id,idx,date,qty
0,1000012,1122,1122_1000012,2019-03-01,0.0
1,1000012,1122,1122_1000012,2019-03-02,15.0
2,1000012,1122,1122_1000012,2019-03-03,15.0
3,1000012,1122,1122_1000012,2019-03-04,0.0
4,1000012,1122,1122_1000012,2019-03-05,0.0
...,...,...,...,...,...
223627,A0000163,BJ17,BJ17_A0000163,2019-08-27,7.5
223628,A0000163,BJ17,BJ17_A0000163,2019-08-28,7.5
223629,A0000163,BJ17,BJ17_A0000163,2019-08-29,17.5
223630,A0000163,BJ17,BJ17_A0000163,2019-08-30,0.0
